### 직방의 원룸 매물정보 수집
- 절차
    - 동 이름으로 위도, 경도 수집
    - 위도, 경도를 geohash로 변환
    - geohash(영역) > 매물 아이디
    - 매물 아이디를 이용해서 매물 정보를 수집

In [9]:
# !pip install geohash2 : geohash2 설치

In [8]:
import requests, json
import pandas as pd
import geohash2

In [ ]:
# 1. 동이름 > 위도, 경도

In [13]:
addr = '강남구 역삼동'
url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
response = requests.get(url)
data = response.json()["items"][0]
lat, lng = data["lat"], data["lng"]
lat, lng

(37.495365142822266, 127.03306579589844)

In [14]:
# 2. 위도, 경도 > geohash

In [15]:
# precision이 커질수록 영역이 작아짐
geohash = geohash2.encode(lat, lng, precision=5)
geohash

'wydm6'

In [ ]:
# 3. geohash > 매물 아이디

In [18]:
url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
response = requests.get(url)
response

<Response [200]>

In [20]:
items = response.json()["items"]
len(items), items[:2]

(2344,
 [{'lat': 37.48565209529541, 'lng': 127.01361043630392, 'item_id': 32819635},
  {'lat': 37.485664670596115, 'lng': 127.01300278169049, 'item_id': 32838798}])

In [21]:
ids = [item["item_id"] for item in items]
ids[:5]

[32819635, 32838798, 32872608, 32838700, 32767020]

In [ ]:
# 4. 매물 아이디 > 매물 정보

In [31]:
url = 'https://apis.zigbang.com/v2/items/list'
params = { # POST 방식일때는 params가 필요. GET 방식일때는 정보가 url에 있기 때문에 필요없음.
    "domain" : "zigbang",
    "withCoalition" : "true",
    "item_ids" : ids[:900]
} # 어떤 파라미터가 필요한 지는 구글 개발자 도구(F12)의 Payload에서 찾아볼 수 있다.

response = requests.post(url, params)
response

<Response [200]>

In [26]:
items = response.json()["items"]
columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
df = pd.DataFrame(items)[columns]
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
898,32844154,월세,100,100,29.75,서울시 강남구 역삼동,9
899,32656329,전세,24000,0,59.50,서울시 강남구 역삼동,10


In [30]:
# 참고 : max, row, max column 설정
# pd.DataFrame(items).tail(2) # columns가 많아서 중간에 생략되서 보인다.
# pd.options.display.max_columns = 50
# pd.DataFrame(items).tail(2) # max columns가 증가하여서 생략되지 않는다.

In [40]:
%%writefile zigbang.py
# %%writefile (파일이름) -- 파일로 만들어줌.(쥬피터) 단, 주석 포함하여 맨 윗줄에서 실행하여야 한다.
# colab 에서도 사용 가능.
# 함수 만들기
import requests
import pandas as pd
import geohash2

def oneroom(addr):
    url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸'
    response = requests.get(url)
    data = response.json()["items"][0]
    lat, lng = data["lat"], data["lng"]
    
    geohash = geohash2.encode(lat, lng, precision=5)
    
    url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
    # 참고: 문자열 길이가 너무 길어 '\'를 사용한다면
    # 함수 안에서 사용한다고 해도 들여쓰기는 하면 안된다. 들여쓰기(tab) 또한 문자로 확인하기 때문.
    response = requests.get(url)
    items = response.json()["items"]
    ids = [item["item_id"] for item in items]
    
    url = 'https://apis.zigbang.com/v2/items/list'
    params = { 
        "domain" : "zigbang",
        "withCoalition" : "true",
        "item_ids" : ids[:900]
    }
    response = requests.post(url, params)

    items = response.json()["items"]
    columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
    return pd.DataFrame(items)[columns]

Overwriting zigbang.py


In [34]:
addr = "마포구 합정동"
df = oneroom(addr)
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
742,32862490,전세,43000,0,31.74,서울시 마포구 중동,5
743,32866975,월세,3000,90,49.59,서울시 마포구 중동,2


In [38]:
df_filtered = df[df["address1"].str.contains(addr)].reset_index(drop=True)
df_filtered

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32662713,전세,17700,0,39.67,서울시 마포구 합정동,0
1,32668759,전세,17700,0,39.67,서울시 마포구 합정동,0
2,32726690,전세,19900,0,36.28,서울시 마포구 합정동,0
3,32542700,월세,1500,95,49.59,서울시 마포구 합정동,7
4,32513282,전세,5000,0,23.14,서울시 마포구 합정동,5
...,...,...,...,...,...,...,...
95,32794268,월세,1000,90,59.50,서울시 마포구 합정동,2
96,32794303,월세,3000,80,59.50,서울시 마포구 합정동,2
97,32804404,월세,2000,85,61.24,서울시 마포구 합정동,2
98,32812101,월세,3000,80,61.24,서울시 마포구 합정동,2
